In [1]:
import os
from glob import glob
import pandas as pd
import json

In [2]:
root = "/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/라벨링데이터/단일경구약제_5000종"
root

'/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/라벨링데이터/단일경구약제_5000종'

In [3]:
df = pd.read_csv("./code2idx.csv")
df

,item_seq,idx,drug_n,name,group
0,200402063,0,K-014967,맥코이정 170mg/병,TS_42_단일
1,200401786,1,K-014870,액시딘캡슐,TS_42_단일
2,200402637,2,K-015266,아모디핀정 5mg,TS_42_단일
3,200402487,3,K-015211,록소론정,TS_42_단일
4,200402643,4,K-015269,록소드펜정,TS_42_단일
...,...,...,...,...,...
3965,200500225,3965,K-016531,카비엑스엘서방정,TS_45_단일
3966,200500578,3966,K-016742,자나디핀정(레르카니디핀염산염),TS_45_단일
3967,200500288,3967,K-016567,리바로정 2mg,TS_45_단일
3968,200500676,3968,K-016790,암로딘정,TS_45_단일


In [4]:
item_codes = [199303108,200704590,200400485]
for code in item_codes:
    print(df[df["item_seq"] == code])

       item_seq   idx    drug_n        name
2515  199303108  2515  K-004378  타이레놀정500mg
      item_seq  idx    drug_n          name
423  200704590  423  K-021026  펠루비정(펠루비프로펜)
       item_seq   idx    drug_n  \
2659  200400485  2659  K-014267   

                                                   name  
2659  이지엔6프로연질캡슐(덱시부프로펜)(수출명:DAUFENSoftCapsuleSPEEDFEN)  


In [5]:
tl_folders = {"TL_3_단일" : "K-004378_json", "TL_51_단일" : "K-021026_json", "TL_40_단일" : "K-014267_json"}

path = root+f"/TL_3_단일/K-004378_json"
json_file = glob(path+"/*.json")[0]

with open(json_file, "rb") as f:
    data = json.load(f)
    
data

{'images': [{'file_name': 'K-004378_0_2_0_0_70_120_200.png',
   'width': 976,
   'height': 1280,
   'imgfile': 'K-004378_0_2_0_0_70_120_200.png',
   'drug_N': 'K-004378',
   'drug_S': '정상알약',
   'back_color': '연회색 배경',
   'drug_dir': '앞면',
   'light_color': '전구색',
   'camera_la': 70,
   'camera_lo': 120,
   'size': 200,
   'dl_idx': '4377',
   'dl_mapping_code': 'K-004378',
   'dl_name': '타이레놀정500mg',
   'dl_name_en': 'Tylenol Tab. 500mg',
   'img_key': 'http://connectdi.com/design/img/drug/147427630838400132.jpg',
   'dl_material': '아세트아미노펜',
   'dl_material_en': 'Acetaminophen',
   'dl_custom_shape': '정제, 저작정',
   'dl_company': '(주)한국얀센',
   'dl_company_en': 'Janssen Korea',
   'di_company_mf': '',
   'di_company_mf_en': '',
   'item_seq': 199303108,
   'di_item_permit_date': '20010404',
   'di_class_no': '[01140]해열.진통.소염제',
   'di_etc_otc_code': '일반의약품',
   'di_edi_code': '646900720',
   'chart': '백색의 장방형 필름코팅정제',
   'drug_shape': '장방형',
   'thick': 6.5,
   'leng_long': 17.5,
   'le

In [6]:
code = data["images"][0]["item_seq"]


In [7]:
df[df["item_seq"] == code]

,item_seq,idx,drug_n,name
2515,199303108,2515,K-004378,타이레놀정500mg


In [8]:
idx = df[df["item_seq"] == code]["idx"].iloc[0]
idx

2515

In [9]:
height = data["images"][0]["height"]
height

1280

In [10]:
width = data["images"][0]["width"]
width

976

In [11]:
data["annotations"][0]["bbox"]

[354, 441, 250, 413]

In [12]:
# yolo 포맷은 객체의 중심좌표와 너비, 높이를 이미지의 너비와 높이로 정규화한 값을 사용
# 좌상단좌표(x,y)와 너비, 높이 정보를 가지고 Yolo포맷으로 바꾸는 파이썬 코드

def convert_to_yolo(x, y, width, height, image_width, image_height):
    # 객체의 중심 좌표 계산
    center_x = x + width / 2
    center_y = y + height / 2
    
    # 좌표와 너비, 높이를 이미지 크기로 정규화
    normalized_center_x = center_x / image_width
    normalized_center_y = center_y / image_height
    normalized_width = width / image_width
    normalized_height = height / image_height
    
    # Yolo 포맷으로 변환된 값을 반환
    return normalized_center_x, normalized_center_y, normalized_width, normalized_height

In [13]:
x,y,w,h = data["annotations"][0]["bbox"]

center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width,height)

In [14]:
center_x

0.49077868852459017

In [15]:
print(data["images"][0]["file_name"])
filename, _ = data["images"][0]["file_name"].split(".")
saved_folder = os.getcwd()+"/labels"
if not os.path.exists(saved_folder):
    os.mkdir(saved_folder)
print(saved_folder)

K-004378_0_2_0_0_70_120_200.png
/home/j-j10b102/labels


In [16]:


for key,value in tl_folders.items():
    path = root+f"/{key}/{value}"
    for json_file in glob(path+"/*.json"):

        with open(json_file, "rb") as f:
            data = json.load(f)
        
        code = data["images"][0]["item_seq"]
        
        idx = df[df["item_seq"] == code]["idx"].iloc[0]
        image_height = data["images"][0]["height"]
        image_width = data["images"][0]["width"]
        x,y,w,h = data["annotations"][0]["bbox"]
        
        center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width,height)
        
#         print(idx, center_x, center_y, normalized_w, normalized_h)
        filename, _ = data["images"][0]["file_name"].split(".")
        saved_path = saved_folder +f"/{filename}.txt"
        with open(saved_path, "w") as f:
            f.write(str(idx)+" ")
            f.write(str(center_x)+" ")
            f.write(str(center_y)+" ")
            f.write(str(normalized_w)+" ")
            f.write(str(normalized_h))
            